
# Callin Switzer
Read in all csv files from individual trials for heavy/light flower experiment, and 
combine into a single long dataset

Also, clean the heavy/light flower metadata


In [4]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import os
import pandas as pd
import time
import re
from datetime import datetime
import sys
import csv
import seaborn as sns
from collections import Counter

print(sys.version, "\n")

now = datetime.now()
print("last run on " + str(now))

3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, 13:04:09) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)] 

last run on 2017-12-07 15:59:08.481536


In [8]:
# define directories
baseDir = os.getcwd()
dataDir = '/Users/cswitzer/Dropbox/SonicationBehavior/SonBehData/SonicationBehavior_HeavyLight/'

In [9]:
baseDir

'/Users/cswitzer/Documents/GitRepos/BeeSonicationLearning'

### Clean bee metadata

In [36]:
metadataFile = "Raw_BeeMetadata_HeavyLightFlowers.csv"

# read in data
hlf = pd.read_csv(dataDir + metadataFile, header = 0, index_col = None)

In [37]:
hlf.head()

,dateFrozenOrMarked,trainFlowerPrepTime,trainStartTime,Hive,FlowerType1,FlowerType2,FlowerSwitchTime,switchRewardNum,trainingThreshold,BeeColorNum,IT,DateIT_Recorded,notes,trainNumRewardVisits,trialEndTime,trainingFile,notes.1
0,26 Sept 2016,1418.0,NaN,3.0,solanum-glued,NaN,NaN,NaN,none,orange,3.71,NaN,NaN,NaN,NaN,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN
1,26 Sept 2016,NaN,NaN,3.0,weighted-solanum-glued,NaN,NaN,NaN,none,orange,3.71,NaN,NaN,NaN,NaN,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,same bee as above
2,26 Sept 2016,1642.0,NaN,3.0,weighted-solanum-glued,NaN,NaN,NaN,none,gold,4.13,NaN,NaN,NaN,1651.0,NaN,NaN
3,26 Sept 2016,1653.0,1703.0,4.0,sham-weighted_solanum_glued,NaN,NaN,NaN,NaN,white,3.38,NaN,NaN,NaN,1712.0,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN
4,26 Sept 2016,1713.0,NaN,3.0,weighted,sham,NaN,24,none,gold,4.13,NaN,NaN,NaN,1732.0,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,"at reward #24, the bee started buzzing on a no..."


In [38]:
# discard rows where I have noted that they should be discarded
discc = []
for ii in range(len(hlf)):
    sst = "".join(np.array(hlf.loc[ii,:]).astype(str)).lower()
    if "discard" in sst:
        discc.append(True)
    else:
        discc.append(False)

hlf["discardRow"] = discc

In [52]:
hlf2= hlf.loc[~hlf.discardRow, :]
hlf2.reset_index(inplace = True, drop=True)


In [53]:
# remove preliminary trials
rightFlowerType = [hlf2.FlowerType1[ii] in ["sham", "weighted"] for ii in range(len(hlf2))]

hlf3 = hlf2.loc[rightFlowerType, ]
hlf3.reset_index(inplace = True, drop=True)

In [54]:
hlf3

,dateFrozenOrMarked,trainFlowerPrepTime,trainStartTime,Hive,FlowerType1,FlowerType2,FlowerSwitchTime,switchRewardNum,trainingThreshold,BeeColorNum,IT,DateIT_Recorded,notes,trainNumRewardVisits,trialEndTime,trainingFile,notes.1,discardRow
0,26 Sept 2016,1713.0,NaN,3.0,weighted,sham,NaN,24,none,gold,4.13,NaN,NaN,NaN,1732.0,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,"at reward #24, the bee started buzzing on a no...",False
1,27 Sept 2016,1032.0,1052.0,4.0,weighted,sham,1055,25,NaN,1,4.07,NaN,switchNum is first rew num after switching,NaN,1059.0,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,frozen,False
2,27 Sept 2016,1106.0,1155.0,4.0,sham,weighted,1157,35,NaN,2,4.42,NaN,NaN,NaN,1201.0,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN,False
3,28 Sept 2016,826.0,832.0,3.0,sham,weighted,835,32,none,3,4.27,NaN,NaN,NaN,838.0,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN,False
4,28 Sept 2016,841.0,845.0,3.0,weighted,sham,847,45,none,4,4.27,NaN,sham flower not positioned very well (needle s...,NaN,NaN,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN,False
5,28 Sept 2016,926.0,928.0,3.0,sham,weighted,939,27,none,5,3.76,NaN,blue dot bee that I marked when she escaped th...,NaN,NaN,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN,False
6,28 Sept 2016,935.0,941.0,3.0,weighted,sham,945,36,none,6,4.55,NaN,NaN,NaN,NaN,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN,False
7,28 Sept 2016,952.0,958.0,3.0,sham,weighted,1001,34,none,7,3.98,NaN,NaN,NaN,NaN,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN,False
8,28 Sept 2016,1129.0,1131.0,4.0,weighted,sham,1133,25,none,8,3.84,NaN,NaN,NaN,NaN,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN,False
9,28 Sept 2016,1140.0,928.0,3.0,sham,weighted,1145,26,none,9,4.17,NaN,NaN,NaN,NaN,/Users/callinswitzer/Dropbox/ExperSummer2016/B...,NaN,False


In [13]:
hlf[hlf['FlowerSwitchTime'].str.contains("discard")]

ValueError: cannot index with vector containing NA / NaN values

In [ ]:
# subset to good trials
rmRows = grep(pattern = "[dD]iscard", tolower(apply(hlf, MARGIN = 1, paste, collapse = ' ')))